In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from zoneinfo import ZoneInfo

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("Play Store Data.csv")

# -------------------------
# CLEAN DATA
# -------------------------

# Installs → numeric
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "")
    .str.replace(",", "")
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Reviews & Rating
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# Size → MB
def size_to_mb(x):
    if isinstance(x, str) and "M" in x:
        return float(x.replace("M", ""))
    return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# Last Updated → datetime
df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")

# -------------------------
# FILTER CONDITIONS
# -------------------------
filtered = df[
    (df["Rating"] >= 4.0) &
    (df["Size_MB"] >= 10) &
    (df["Last Updated"].dt.month == 1)
]

# -------------------------
# TOP 10 CATEGORIES BY INSTALLS
# -------------------------
top10 = (
    filtered.groupby("Category")["Installs"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .index
)

top_df = filtered[filtered["Category"].isin(top10)]

# -------------------------
# AGGREGATE METRICS
# -------------------------
summary = (
    top_df.groupby("Category")
    .agg({
        "Rating": "mean",
        "Reviews": "sum"
    })
    .reset_index()
)

# -------------------------
# TIME CONDITION (3–5 PM IST)
# -------------------------
current_time = datetime.now(ZoneInfo("Asia/Kolkata"))

if 15 <= current_time.hour < 17:

    # -------------------------
    # GROUPED BAR CHART
    # -------------------------
    fig = go.Figure()

    fig.add_bar(
        x=summary["Category"],
        y=summary["Rating"],
        name="Average Rating"
    )

    fig.add_bar(
        x=summary["Category"],
        y=summary["Reviews"],
        name="Total Reviews"
    )

    fig.update_layout(
        title="Top 10 App Categories — Rating vs Reviews",
        barmode="group",
        xaxis_title="Category",
        yaxis_title="Value"
    )

    fig.show()

else:
    print("Chart hidden — Available only between 3 PM and 5 PM IST")


Chart hidden — Available only between 3 PM and 5 PM IST
